# **Exploratory Data Analysis and Feature Engineering on Walmart Weekly Sales Dataset**  
![](https://i.imgur.com/qwvOeHo.jpg)



Walmart is an American Retail, Wholesale and E-commerce business. Sam Walton founded Walmart in 1962 in Rogers, Arkansas. His goal was to help people "Save Money and Live Better" which continues to be Walmart's guiding mission with "Every Day Low Prices(EDLC)" and great service.
- Number 1 ranked [Fortune 500](https://fortune.com/fortune500/) company with [total revenue](https://s2.q4cdn.com/056532643/files/doc_financials/2021/ar/WMT_2021_AnnualReport.pdf) of $559 billion as of Jan 31 2021.
- Publicly listed company, 'WMT' on the New York Stock Exchange(NYSE)
- Each week 220 million customers visit 10,500 stores and clubs under 48 banners in 24 countries and eCommerce websites
- [Largest](https://www.statista.com/statistics/264671/top-50-companies-based-on-number-of-employees/) private employer in the world with more than 2.3 million people employed around the world


Revenue and operations of the company are categorised into three key business segments - Walmart US, Walmart International and Sam's Club.

They operate in three store types or store formats as below.
- Supercenters (general merchandise and grocery, Average store size 178,000 square feet)
- Discount stores (general merchandise and limited grocery, Average size 106,000 square feet)
- Neighbourhood stores (grocery, Average size 42, 000 square feet)




In this notebook, I will explore Walmart Dataset by doing the Exploratory Data Analysis and Feature Engineering.  I will use **Pandas**, **Numpy**, **Matplotlib**, **Seaborn** and **Plotly** to perform exploratory data analysis and gather insights information of dataset. I will do the following

- Install and Import libraries
- Explore the dataset and merge different files as required
- EDA - exploratory data analysis
- Feature Engineering
- Data preparation - Train Val Split, Encoding, Imputing and Scaling
- Select input features
- Define evaluation metrics
- Define baseline model


## Install and import libraries

In [ ]:
#!pip install pandas numpy sklearn opendatasets seaborn matplotlib plotly --upgrade --quiet

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

In [ ]:
# OS and Data libraries
import os
from zipfile import ZipFile

#Data analysis libraries
import numpy as np
import pandas as pd

#Visualisation libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#date formatters
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

#Jupyter notebook visualisation font and figure display parameters
%matplotlib inline

sns.set_style('ticks')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

## Exploring the dataset

### Download and extract data from zip files  

In [ ]:
kaggle_url = "https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data"

In [ ]:
od.download(kaggle_url)

In [ ]:
os.listdir("walmart-recruiting-store-sales-forecasting")

In [ ]:
# Helper function to unzip many ".zip" files in a directory

from zipfile import ZipFile

def unzip_files(path):

    # Get list of files in the directory
    files = os.listdir(path)

    for file in files:                                   #For each file in the directory

        if file.endswith(".zip"):                        # if it is a ".zip" file
            file_path = path+"/"+file                    # get location of the file

            with ZipFile(file_path,"r") as zip_file:     #Open the ".zip" file in read mode
                zip_file.extractall(path)                #extract all files in ".zip" in the location

    return

In [ ]:
dir_path = "./walmart-recruiting-store-sales-forecasting"

#unzip all ".zip" files
unzip_files(dir_path)

os.listdir("walmart-recruiting-store-sales-forecasting")

In [ ]:
import glob

glob.glob(dir_path+"/"+"*.csv")

In [ ]:
train_df = pd.read_csv('./walmart-recruiting-store-sales-forecasting/train.csv')
test_df = pd.read_csv('./walmart-recruiting-store-sales-forecasting/test.csv')
stores_df = pd.read_csv('./walmart-recruiting-store-sales-forecasting/stores.csv')
features_df = pd.read_csv('./walmart-recruiting-store-sales-forecasting/features.csv')
submission_df = pd.read_csv('./walmart-recruiting-store-sales-forecasting/sampleSubmission.csv')


### Explore data in indivdual files

In [ ]:
train_df.info()

In [ ]:
train_df.describe().transpose()

In [ ]:
train_df.astype('object').describe().transpose()

In [ ]:
train_df.isna().sum()

**Train.csv**
- `Store`: Store number from 1 to 45 doesnt indicate any ranking.
- `Dept`: Numbered 1 - 99. There are 81 departments in this dataset. So some stores have much fewer than 81 departments. Stores may have started some departments later
- `Date`: Date corresponding to Thursday of each week (So a week is Friday - Thursday)
- `IsHoliday`: A boolean indicating whether it is a store holiday or not
- `Weekly_Sales`: Sales for the week reported on Thursdays of each week.
- The complete history of 143 weeks is therefore not available for all departments
- No null values in this dataset


In [ ]:
test_df.info()

In [ ]:
test_df.describe().transpose()

In [ ]:
test_df.astype('object').describe().transpose()

In [ ]:
test_df.isna().sum()

**Test.csv**
- `Store`: Store number from 1 to 45 doesnt indicate any ranking.
- `Dept`: Numbered 1 - 99. 81 departments in this dataset. So some stores have much fewer than 81 departments. Stores may have started some departments later
- `Date`: Date corresponding to Thursday of each week (So a week is Friday - Thursday)
- `IsHoliday`: A boolean indicating whether it is a store holiday or not
- 39 forecast weeks
- No null values in this dataset

In [ ]:
stores_df.info()

In [ ]:
stores_df.describe().transpose()

In [ ]:
stores_df['Type'].unique()

In [ ]:
stores_df.isna().sum()

**Stores.csv**
- Each row of data contains the store ID, Store Type and Size of the store.
- `Store`: There are 45 stores
- `Type`: 3 types, A, B and C.
- `Size` of the store is likely to be square feet area
- There are no null values in this dataset.

In [ ]:
features_df.info()

In [ ]:
features_df.describe()

In [ ]:
features_df.sample(5)

**Features.csv**
- Each row corresponding to one week recorded on Thursdays. 10 feature columns, 1 date column and 1 store ID column.
- Feature data starts from 5th Feb 2010, and ends on 26th July 2013. So we - have 1267 days or 3.5 years of data. The stores had 13 holidays during this period
- `MarkDown1, MarkDown2,MarkDown3,MarkDown4,MarkDown5`. There are 5 different types of markdowns. There are missing values for markdowns, CPI and unemployment. The competition also notes that markdowns are only available from Nov-11 and not available for all stores.
- `Temperature` - Likley to be Temperature of the location where the store is located in Fahrenheit
- `CPI` - Likley to be the US Consumer Price Index
- `Fuel Price` - Likely to be fuel price in US Dollars
- `Unemployment` - Likely to be US unemployment numbers



### Merge store and features into train and test data

In [ ]:
merged_train_df = pd.merge(train_df, stores_df, how = "left").merge(features_df, how = 'left')
merged_test_df = pd.merge(test_df, stores_df, how = "left").merge(features_df,how = 'left')
merged_train_df["Date"] = pd.to_datetime(merged_train_df['Date'])
merged_test_df["Date"] = pd.to_datetime(merged_test_df['Date'])

- 536,634 rows ( train = 421,570, test = 115,064) and 14 columns  
- Weekly sales numbers ordered by date for 45 stores and 81 departments ( numbered 1 to 99)
- Grouped/ hierarchical time series i.e, weekly sales  on a particular date each week, for each store and department in each store
- Internal inputs : Store Number, Store Type, Store Size, Dept number , IsHoliday, mark downs
- External inputs:  CPI, Temperature, Unemployment & Fuel Price

## Translating business problem to machine learning

In [ ]:
merged_train_df.columns


**Target variable**: Predict “Weekly_Sales” for dates in the future using current information. Our target variable “Weekly_Sales”, is a series of continuous values.   

**Input variables** : We can categorise the variables as below
- Internal inputs : Store Number, Store Type, Store Size, Dept number , IsHoliday, Markdowns (1 to 5)
- External inputs:  CPI, Temperature, Unemployment & Fuel Price   

  

**Model:**
This problem can be modeled using a single classical **supervised** machine learning **regression** model covering all stores and departments.  

We will use a **regression** model to predict our target variable “Weekly_Sales” which is a **grouped/hierarchical** time series.  

We will explore the following categories of models, engineer features and hypertune parameters to choose a model with the best fit.
- Linear models
- Tree models
- Ensemble models



**Evaluation criteria**. In addition to WMAE, we will also use additional evaluation criteria such as MAE, RMSE and R2

## Feature Engineering

This section has been moved up as some of them are used in EDA too. As we have seen from the correlation analysis, the basic inputs variables seem to have weak predictive power. We can therefore explore if we can create useful features.
This is an iterative process, where new features can be explored over time.

The first basic features created were the date features, the rest were added iteratively during the exploratory data analysis.

### Date Features
We will extract features from the Date. This is the most basic feature engineering required to provide meaningful inputs to the model as they expect numeric inputs instead of a DateTime type.

In [ ]:
def create_date_features(df):

    df["Date"] = pd.to_datetime(df['Date'])
    df['WeekofYear'] = (df['Date'].dt.isocalendar().week).astype('int')
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['Day'] = df['Date'].dt.day

    return df

In [ ]:

merged_train_df = create_date_features(merged_train_df)
merged_test_df = create_date_features(merged_test_df)

## Holiday Features
Holidays have a huge impact on Weekly sales as we discovered from our EDA. We have isolated holidays that have the most impact. Here we will create features for them.

In [ ]:
def create_holiday_features(df):
  df['BlackFriday_Week'] = np.where(np.isin(df['WeekofYear'],np.array([47,48])), 1, 0)
  df['SuperBowl_Week'] = np.where(np.isin(df['WeekofYear'],np.array([5,6])), 1, 0)
  df['Christmas_Season'] = np.where(np.isin(df['WeekofYear'],np.array([49,50,51])), 1, 0)

  return df

In [ ]:
merged_train_df = create_holiday_features(merged_train_df)
merged_test_df = create_holiday_features(merged_test_df)

## Markdown Features

One of the requirements from the business is to analyse the impact of MarkDowns on Weekly Sales, so let us create some markdown features as well.

In [ ]:
def create_markdown_features(df):

  cols_MarkDown = ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']

  has_MarkDown = ~df[cols_MarkDown].isna()
  df['has_MarkDown_info'] = has_MarkDown.prod(axis=1)                                  #set a Markdown flag of Markdown exists
  df['total_MarkDown'] = df[cols_MarkDown].sum(axis=1)                            #Sum of all Markdowns for each row

  for col in cols_MarkDown:
    df["has_"+ col] = np.where(np.isnan(df[col]), 0, 1)                            #set a Markdown flag for each markdown if available
    df["pct_"+ col] = np.where(np.isnan(df[col]), 0, df[col]/df['total_MarkDown']) # pct of each markdown versus total (since we can't do pct of sales in test)



  return df

In [ ]:
merged_train_df = create_markdown_features(merged_train_df)
merged_test_df = create_markdown_features(merged_test_df)

## Performer Features

This set of features was an attempt to overcome the issues with interpretibility and dimensionality of one-hot encoding nominal features such as Store and Dept. But they turned out to be a huge drag on model performance.

In [ ]:
def create_performer_features(df):

  #high sales depts
  df['Top_Depts'] = np.where(np.isin(df['Dept'],np.array([92, 95, 38, 72, 65, 90, 40, 2, 91, 94])), 1, 0)
  #low sales depts
  df['Bottom_Depts'] = np.where(np.isin(df['Dept'],np.array([99, 60, 77, 54, 45, 51, 39, 78, 43, 47])), 1, 0)

  #high sales stores
  df['Top_Stores'] = np.where(np.isin(df['Store'],np.array([20, 4, 14, 13, 2, 10, 27, 6, 1, 39])), 1, 0)
  #low sales stores
  df['Bottom_Stores'] = np.where(np.isin(df['Store'],np.array([30, 36, 7, 29, 16, 38, 3, 44, 33, 5])), 1, 0)

  return df

In [ ]:
merged_train_df = create_performer_features(merged_train_df)
merged_test_df = create_performer_features(merged_test_df)

## Store and Dept Rank Features

`Store` and `Dept` are just numbers which are some form of ID. Let us try to create some features for Store and Department to provide some more meaningful information for our models. We will rank the stores and departments by the average of grouped weekly_sales.

In [ ]:
store_rank_df = merged_train_df.groupby('Store', as_index= False).agg(Avg_Store_Sales = ('Weekly_Sales',lambda x: x.mean())).round(0)
store_rank_df['Store_Rank'] = store_rank_df['Avg_Store_Sales'].rank(ascending= False).astype(int)
store_rank_df.sort_values(by=['Store_Rank'], ascending= True, inplace= True)

In [ ]:
dept_rank_df = merged_train_df.groupby('Dept', as_index= False).agg(Avg_Dept_Sales = ('Weekly_Sales',lambda x: x.mean())).round(0)
dept_rank_df['Dept_Rank'] = dept_rank_df['Avg_Dept_Sales'].rank(ascending= False).astype(int)
dept_rank_df.sort_values(by=['Dept_Rank'], ascending= True, inplace= True)

In [ ]:
def create_rank_features(df):

  df = pd.merge(df, store_rank_df, how='left', on= 'Store')
  df = pd.merge(df, dept_rank_df, how='left', on= 'Dept')

  return df

In [ ]:
merged_train_df = create_rank_features(merged_train_df)
merged_test_df = create_rank_features(merged_test_df)

In [ ]:
merged_train_df.info()

## Exploratory data analysis

### Summary Statistics and Sample Data

In [ ]:
merged_train_df.sample(6)

In [ ]:
merged_train_df.describe()

In [ ]:
merged_test_df.sample(5)

In [ ]:
merged_test_df.describe()

Assumptions:

- Size is the store Size and in square feet
- Temperature is in Farenheit
- Weekly Sales is un US Dollars ($)

- Weekly sales has negative values. This could be because weekly sales = sales  returns - discounts
- 75% of Weekly Sales are less than or equal to \\$20K But the highest Weekly - Sales is of \\$693K. This suggests that the data has extreme outliers.
- Temperature has varied from -2 degree Fahrenheit to 100 degree Fahrenheit
- The other values seem to be reasonable with respect to summary statistics

### Availability of Train and Test Data

In [ ]:
train_avail= merged_train_df.groupby(by="Year", as_index= False)['Store'].agg({'train_count':"count"})
test_avail = merged_test_df.groupby(by="Year", as_index= False)['Store'].agg({'test_count':"count"})
data_avail = pd.merge(train_avail, test_avail, how= 'outer', on='Year').fillna(0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Data Availability')
data_avail.plot(kind='bar', x='Year', ax= axes[0])
axes[1].set_title("Train data by month")
sns.countplot(data= merged_train_df, x="Month", hue="Year",ax=axes[1] )
sns.despine();

- Train data is available from 5, Feb 2010 to 26 October 2012. 143 weeks of data
- Test data starts from 02, November, 2012 and is available upto 26-July-2013





### Analysis of Stores by Type and Size

In [ ]:
fig,axes = plt.subplots(1,2,figsize= (18,6))
sns.countplot(data= stores_df,  x = "Type", ax=axes[0])
axes[0].set_title("Number of stores by Type")
sns.stripplot(x="Type", y= "Size", data= stores_df, ax = axes[1])
axes[1].set_title("Stores by Size")
sns.despine();

- Type A stores (Supercenters) are the largest of the three types (with a few exceptions)
- Type C stores (neighbourhood stores) are the smallest in size  
- There are more stores of Type A (22) and very few stores of Type C(6)
- A few Type A (Supercenters) and B (discount stores) also have sizes similar to Type C (neighbourhood) stores

### Weekly Sales

In [ ]:
fig, ax = plt.subplots(figsize = (20,6))
merged_train_df.plot(y='Weekly_Sales', x='Date', ax=ax)
plt.title("Train Data: Weekly Sales")
sns.despine();

It is hard to single out trends in this graph. Let us try to smoothen the data to isolate broader trends. Seaborn does this automatically as below.

In [ ]:
fig, ax = plt.subplots(figsize = (20,6))
sns.lineplot(data=merged_train_df, x="Date", y="Weekly_Sales")
plt.title("Train Data: Smoothened Weekly Sales Seasonality and Trends")

date_format = DateFormatter("%b-%y")
ax.xaxis.set_major_formatter(date_format)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
sns.despine();

There seem to be broad trends around Nov-Dec each year. Let us try to isolate some more patterns.

### Weekly Sales by Month

In [ ]:
fig, axes = plt.subplots(figsize = (18,6))
fig.suptitle(" Train Data: Weekly Sales by Month of the Year")
sns.violinplot(data= merged_train_df, y='Weekly_Sales', x = "Month")
sns.despine();

Weekly sales are reported on the Thursday of each week. We see strong seasonality around the same time each year. Let us explore this further

### Weekly Sales by Week of the Year

In [ ]:
fig, axes = plt.subplots(figsize = (18,6))
fig.suptitle(" Train Data: Weekly Sales by Week of the Year")
sns.boxplot(data= merged_train_df, y='Weekly_Sales', x = "WeekofYear")
sns.despine();

### Weekly Sales by Store

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

grouped = merged_train_df.groupby('Store')
for key, group in grouped:
    group.plot(ax=ax, kind='line', x='Date', y='Weekly_Sales', label=key)

date_format = DateFormatter("%b-%y")
ax.xaxis.set_major_formatter(date_format)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))

plt.title("Train Data: Weekly Sales by Store")
plt.legend(fontsize='small', ncol=4,handleheight=2.4, labelspacing=0.05, frameon= False)
sns.despine()

Not all stores seem to have the same peaks. Let us now explore departments.

#### Weekly Sales by Dept

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

grouped = merged_train_df.groupby('Dept')
for key, group in grouped:
    group.plot(ax=ax, kind='line', x='Date', y='Weekly_Sales', label=key)

date_format = DateFormatter("%b-%y")
ax.xaxis.set_major_formatter(date_format)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))

plt.title("Train Data: Weekly Sales by Dept")
plt.legend(fontsize='xx-small', ncol=4,handleheight=2.4, labelspacing=0.05, frameon= False)
sns.despine()

Individual departments seem to have different patterns too. Let us now see a single store with a couple of departments.

#### Weekly Sales by Select store and Dept

In [ ]:
single_store_dept_mask1 = (merged_train_df.Store == 1) & (merged_train_df.Dept == 1)
single_store_dept_mask2 = (merged_train_df.Store == 1) & (merged_train_df.Dept == 72)

In [ ]:
fig, ax = plt.subplots(figsize = (20,6))
merged_train_df[single_store_dept_mask1].plot(y='Weekly_Sales', x='Date',label = 'Dept1', ax=ax)
merged_train_df[single_store_dept_mask2].plot(y='Weekly_Sales', x='Date', label = 'Dept72', ax=ax)
plt.title("Train Data: Weekly Sales train data for Store 1,  Dept 1 & 72")
sns.despine();

Therefore it will be important to retain Store and Department as features.

### How do holidays effect Weekly Sales?

In [ ]:
fig = px.histogram(train_df,
                  x='Weekly_Sales',
                   marginal='box',
                   color = 'IsHoliday',
                   color_discrete_sequence=['green', 'grey'],
                   nbins = 50,
                   title='Distribution of Weekly Sales by Holiday')
fig.update_layout(bargap=0.1)
fig.show()

- Weekly sales for weeks with holidays are quite similar to weeks which have no holidays
- But there are definitely some weeks with holidays which see blockbuster sales.
- This is probably because stores run promotions (with markdowns) during certain holidays.
Walmart is famous for its annual mega promos for Black Friday which is the Friday after ThanksGiving.  
Let us now analyse these further.  

![](https://i.imgur.com/4Q12u2a.png)

### Do larger stores sell more?

In [ ]:
fig = px.histogram(merged_train_df,
                  x='Weekly_Sales',
                  marginal='box',
                  color = 'Type',
                   color_discrete_sequence=['blue','green', 'grey'],
                   nbins = 50,
                  title='Distribution of Weekly Sales by Store Type')
fig.update_layout(bargap=0.1)
fig.show()

- The larger size stores, Type A have higher Weekly Sales than Type B and C Stores
- However Type B stores have most of the outliers or blockbuster sales. There are most likely the discount stores running holiday promotions with Markdowns(discounts)  
Let us now explore these blockbuster sales further

### When do the blockbuster sales occur?

In [ ]:
blockbusters = merged_train_df.Weekly_Sales > 220000
blockbusters_df = merged_train_df[blockbusters].sort_values(by='Date', ascending= True)
blockbusters_days = [pd.to_datetime(day).strftime("%Y %B %d") for day in blockbusters_df.Date.unique()]
blockbusters_days

- Holidays which have an impact for certain: Week of Super Bowl, Black Friday, Christmas Eve.
- Possibly Christmas Season from ThanksGiving to Christmas  
- Labour day doesn't seem to figure in this list  
- It's also interesting to see blockbuster sales during the events of Occupy Wall Street protests, Hurricane Irene and the US primaries

### Which departments sell more and which ones sell the least?

In [ ]:
fig, ax = plt.subplots(figsize = (22,6))
fig.suptitle("Weekly Sales by Department")
sns.boxplot(data= merged_train_df, y='Weekly_Sales', x = 'Dept', color = 'cyan')
sns.despine();

In [ ]:
#Top 10 Departments
depts = merged_train_df.groupby(by=('Dept'), as_index= False)['Weekly_Sales'].mean().sort_values(by=('Weekly_Sales'), ascending= False)
top_10_depts = depts.head(10)
list(top_10_depts.Dept)

In [ ]:
#Bottom 10 Departments
bottom_10_depts = depts.tail(10)
list(bottom_10_depts.Dept)

### Which stores have sold the most and which ones sold the least?

In [ ]:
fig, ax = plt.subplots(figsize = (22,6))
fig.suptitle("Weekly Sales by Store")
sns.boxplot(data= merged_train_df, y='Weekly_Sales', x = "Store", color="skyblue")
sns.despine();

In [ ]:
#Top 10 Stores
stores = merged_train_df.groupby(by=('Store'), as_index= False)['Weekly_Sales'].mean().sort_values(by=('Weekly_Sales'), ascending= False)
top_10_stores = stores.head(10)
list(top_10_stores.Store)

In [ ]:
#Bottom 10 Stores
bottom_10_stores = stores.tail(10)
list(bottom_10_stores.Store)

### How do MarkDowns and Holidays effect Weekly Sales?

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (18,6))
fig.suptitle("Effect of Holidays and Markdowns on Weekly Sales")
sns.barplot(data= merged_train_df, x="has_MarkDown_info", y="Weekly_Sales", ax= axes[0])
sns.barplot(data=merged_train_df, x="IsHoliday", y="Weekly_Sales",  ax = axes[1])
sns.despine();

Markdowns and Holidays seem to be important as well. But Markdowns are going to be tricky since this information is not present consistently according to the information we have on the competition.

## Data preparation

### Train, Val and Test sets

Splitting available data into train, val and test is an important step before training a model on the data and using the model parameters to using the model in the real world for prediction.

- **train** : used to fit our model (get the parameters based on our features)
- **val** : used to tune hyperparameters ( regularise for bias and variance)
- **test**: evaluate model performance close to real world/future

Therefore it is important that the val and test sets are as close to the real world as possible. So experiment with your train_val_test splits.

Key observations to prevent **Data Leakage**
- Split data before transforming data (encoding, imputing, scaling)
- Since this is a time-series dataset , ensure that we order the data by Date and their respective heirarchies Store and Dept before split. Otherwise we will be using data from the future to train our model.
- If using `sklearn` set `shuffle` to `False` using information from the future

In [ ]:
#Sort dataset by Dates, Store and Dept
merged_train_df.sort_values(by=['Date','Store','Dept'], inplace= True)
merged_test_df.sort_values(by=['Date','Store','Dept'], inplace= True)

#### Identify Numeric and Categorical Data

In [ ]:
merged_train_df.columns

In [ ]:
inputs_colums = ['Store', 'Dept', 'Date','IsHoliday', 'Type', 'Size',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'WeekofYear', 'Month',
       'Year', 'Day', 'BlackFriday_Week', 'SuperBowl_Week', 'Christmas_Season',
       'has_MarkDown_info', 'total_MarkDown', 'has_MarkDown1', 'pct_MarkDown1',
       'has_MarkDown2', 'pct_MarkDown2', 'has_MarkDown3', 'pct_MarkDown3',
       'has_MarkDown4', 'pct_MarkDown4', 'has_MarkDown5', 'pct_MarkDown5',
       'Top_Depts', 'Bottom_Depts', 'Top_Stores', 'Bottom_Stores',
       'Avg_Store_Sales', 'Store_Rank', 'Avg_Dept_Sales', 'Dept_Rank']

In [ ]:
import numpy as np

In [ ]:
numeric_cols = merged_train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
categorical_cols = merged_train_df.select_dtypes('object').columns.tolist()

In [ ]:
print(list(numeric_cols))

In [ ]:
print(list(categorical_cols))

#### Impute Numerical Data

In [ ]:
missing_counts = merged_train_df[numeric_cols].isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

In [ ]:
numeric_cols_for_test = ['Store', 'Dept', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'WeekofYear', 'Month', 'Year', 'Day', 'BlackFriday_Week', 'SuperBowl_Week', 'Christmas_Season', 'has_MarkDown_info', 'total_MarkDown', 'has_MarkDown1', 'pct_MarkDown1', 'has_MarkDown2', 'pct_MarkDown2', 'has_MarkDown3', 'pct_MarkDown3', 'has_MarkDown4', 'pct_MarkDown4', 'has_MarkDown5', 'pct_MarkDown5', 'Top_Depts', 'Bottom_Depts', 'Top_Stores', 'Bottom_Stores', 'Avg_Store_Sales', 'Store_Rank', 'Avg_Dept_Sales', 'Dept_Rank']


In [ ]:
missing_counts = merged_test_df[numeric_cols_for_test].isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

In [ ]:
# Fill missing values with zero
merged_train_df = merged_train_df.fillna(value=0)

In [ ]:
merged_test_df=merged_test_df.fillna(value=0)

In [ ]:
missing_counts = merged_test_df[numeric_cols_for_test].isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

In [ ]:
missing_counts =merged_train_df[numeric_cols].isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

#### Encode Categorical Columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [ ]:
# 2. Fit the encoder to the categorical colums
encoder.fit(merged_train_df[categorical_cols])

In [ ]:
# 2. Fit the encoder to the categorical colums
encoder.fit(merged_test_df[categorical_cols])

In [ ]:
merged_train_df[categorical_cols].nunique()

In [ ]:
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
len(encoded_cols)

In [ ]:
# 4. Transform and add new one-hot category columns
merged_train_df[encoded_cols] = encoder.transform(merged_train_df[categorical_cols])

In [ ]:
# 4. Transform and add new one-hot category columns
merged_test_df[encoded_cols] = encoder.transform(merged_test_df[categorical_cols])

In [ ]:
merged_train_df[:3]

In [ ]:
merged_test_df[:3]

In [ ]:
merged_train_df.columns

In [ ]:
input_col = ['Store', 'Dept', 'Date', 'IsHoliday', 'Type', 'Size',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'WeekofYear', 'Month',
       'Year', 'Day', 'BlackFriday_Week', 'SuperBowl_Week', 'Christmas_Season',
       'has_MarkDown_info', 'total_MarkDown', 'has_MarkDown1', 'pct_MarkDown1',
       'has_MarkDown2', 'pct_MarkDown2', 'has_MarkDown3', 'pct_MarkDown3',
       'has_MarkDown4', 'pct_MarkDown4', 'has_MarkDown5', 'pct_MarkDown5',
       'Top_Depts', 'Bottom_Depts', 'Top_Stores', 'Bottom_Stores',
       'Avg_Store_Sales', 'Store_Rank', 'Avg_Dept_Sales', 'Dept_Rank',
       'Type_A', 'Type_B', 'Type_C']

In [ ]:
target_col = [ 'Weekly_Sales']

In [ ]:
len(merged_train_df)

In [ ]:
merged_test_df

In [ ]:
sample_merged_train_df =merged_train_df[0:29570]

In [ ]:
sample_merged_train_df

In [ ]:
#Split with shuffle False since this is a TimeSeries data set
from sklearn.model_selection import train_test_split
# 90:10 split
inputs_train_df, inputs_val_df, y_train, y_val = train_test_split(sample_merged_train_df[input_col], sample_merged_train_df[target_col], test_size = 0.10, random_state = 42, shuffle= False)

#default split
#inputs_train_df, inputs_val_df, y_train, y_val = train_test_split(merged_train_df[input_col], merged_train_df[target_col], random_state = 42, shuffle= False)

In [ ]:
#Split Dates for plots by similar size
#train_size = int(0.70 * len(merged_train_df))
train_size = len(inputs_train_df)
train_dates, val_dates = sample_merged_train_df[:train_size]['Date'], sample_merged_train_df[train_size:]['Date']

In [ ]:
sample_merged_test_df = merged_test_df[0:13064]

In [ ]:
inputs_test_df = sample_merged_test_df.copy()

In [ ]:
inputs_train_df

In [ ]:
inputs_val_df

## Feature Selection

We may have several input features or variables, but selecting fewer and right input variables will make a huge difference in the predictive power or model outcome. Choosing many variables will lead to complex models with low predictive power and several problems such as
- Increased noise(variables that don't have any explanatory or predictive power)
- Increased dimensionality ( many features which are difficult to interpret and thinly distributed predictive power)
- Underfitting and overfitting

There are many feature selection techniques such as PCA, SVD, autoencoders and Lasso too. We will make some intuition based selection for now and chalk this up for future work.

In [ ]:
new_input_cols = ['Store_Rank', 'Dept_Rank', 'Size', 'Type_A', 'Type_B', 'Type_C',
       'IsHoliday', 'WeekofYear', 'Month', 'Day', 'Year', 'BlackFriday_Week',
       'SuperBowl_Week', 'Christmas_Season', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'Fuel_Price', 'CPI',
       'Unemployment', 'Temperature']

In [ ]:
X_train = inputs_train_df[new_input_cols].copy()

In [ ]:
X_val = inputs_val_df[new_input_cols].copy()

In [ ]:
X_train = inputs_train_df[new_input_cols].copy()
X_val = inputs_val_df[new_input_cols].copy()
X_test = inputs_test_df[new_input_cols].copy()

In [ ]:
X_train.columns

In [ ]:
X_train

In [ ]:
X_val

In [ ]:
X_test 

## Evaluation Metrics

A function to compute Weighted Mean Absolute error(WMAE) as required by the competition is defined. A function to evaluate based on other metrics is also added.

In [ ]:
def weighted_mean_absolute_error(df, y_actuals, y_preds):

  if 'IsHoliday' in df.columns:
    weights = df['IsHoliday'].apply(lambda x: 5 if x ==1 else 1)
  elif 'IsHoliday_1' in df.columns:
    weights = df['IsHoliday_1'].apply(lambda x: 5 if x==1 else 1)

  wmae = (np.sum(abs(y_actuals-y_preds)*weights))/np.sum(weights)

  return wmae

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def model_eval(df, y_actuals, y_preds):

    wmae = round(weighted_mean_absolute_error(df, y_actuals, y_preds),2)
    mae = round(mean_absolute_error(y_actuals, y_preds),2)
    rmse = round(mean_squared_error(y_actuals, y_preds, squared= False),2)
    r2 = round(r2_score(y_actuals, y_preds),2)

    n = len(df)
    p = len(df.columns)
    adj_r2 = round(1-(1-r2)*(n-1)/(n-p-1),2)

    return {'wmae': wmae,
            'mae': mae,
            'rmse': rmse,
            'adj_r2': adj_r2,
            'r2': r2
            }

## Baseline Model

Let's create a simple baseline model to evaluate our models. Here we will use a grouped average, average sales by each store and dept during a month.

In [ ]:
train_preds_grouped_mean = merged_train_df.groupby(['Store','Dept','Month']).Weekly_Sales.transform('mean')
model_eval(merged_train_df, merged_train_df.Weekly_Sales, train_preds_grouped_mean)

Any model we train from now on should give us a better training score than the WMAE of 2120.